## Подключаем библиотеки

In [1]:
import pandas as pd

from sklearn.impute import SimpleImputer as Imputer
from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

### Загружаем датасет

In [2]:
df1 = pd.read_csv('Information and Metrics.csv')
df2 = pd.read_csv('Multi-BBL.csv')

df = pd.concat([df1, df2])
df.head()

,Order,Property Id,Property Name,Parent Property Id,Parent Property Name,BBL - 10 digits,"NYC Borough, Block and Lot (BBL) self-reported",NYC Building Identification Number (BIN),Address 1 (self-reported),Address 2 (self-reported),...,Annual Maximum Demand (kW),Annual Maximum Demand (MM/YYYY),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),Water Use (All Water Sources) (kgal),Water Use Intensity (All Water Sources) (gal/ft²),Water Required?,Generation Date,DOF Benchmarking Submission Status
0,1,4593574,The Argonaut Building,NaN,NaN,1010287502.0,1010287502,1024898,224 West 57th St,NaN,...,NaN,NaN,732.4,76.3,656.1,3635.5,21.46,Not found,2018-02-14,Not found
1,3,2967701,Cathedral Preparatory Seminary,NaN,NaN,4018720007.0,4-01872-0007,4046340,56-25 92nd Street,NaN,...,NaN,NaN,164.5,109.9,54.6,102.9,1.09,Not found,2018-02-14,Not found
2,4,4898531,The Nomad Hotel,NaN,NaN,1008290050.0,1-00829-0050,1080710,1170 Broadway,NaN,...,NaN,NaN,1150.2,438.0,712.3,10762.6,86.10,Not found,2018-02-14,Not found
3,5,2917939,10 West 27 Street Corp,NaN,NaN,1008280053.0,1-00828-0053,1015657,1155 Broadway,NaN,...,NaN,NaN,273.4,98.2,175.3,790.1,15.80,Not found,2018-02-14,Not found
4,6,3878205,Westbury Realty,NaN,NaN,1000650024.0,1-00065-0024,1001105,24 John Street,NaN,...,NaN,NaN,90.7,0.0,90.7,143.0,2.86,Not found,2018-02-14,Not found


### Удаляем ненужные столбцы

In [3]:
df.drop(['DOF Gross Floor Area (ft²)', 'Water Use (All Water Sources) (kgal)', 
         'Water Use Intensity (All Water Sources) (gal/ft²)', 'Property Name',
         'Street Number', 'Street Name', 'Address 1 (self-reported)', 'Order',
         'Water Required?', 'DOF Benchmarking Submission Status'], axis=1, inplace=True)

df.drop(['BBL - 10 digits', 'NYC Borough, Block and Lot (BBL) self-reported',
         'NYC Building Identification Number (BIN)'], axis=1, inplace=True)

In [5]:
# Функция для расчета количества пустых значений
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * mis_val / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [7]:
missing_df = missing_values_table(df)
missing_df

Your selected dataframe has 47 columns.
There are 38 columns that have missing values.


,Missing Values,% of Total Values
Propane Use (kBtu),34685,100.0
District Hot Water Use (kBtu),34678,100.0
District Chilled Water Use (kBtu),34655,99.9
Fuel Oil #1 Use (kBtu),34650,99.9
Diesel #2 Use (kBtu),34614,99.8
Address 2 (self-reported),34224,98.7
Fuel Oil #5 & 6 Use (kBtu),34201,98.6
District Steam Use (kBtu),33105,95.4
Parent Property Name,32706,94.3
Parent Property Id,32706,94.3


In [8]:
# Удаляем столбцы где больше 50% пустых значений
missing_columns = list(missing_df[missing_df['% of Total Values'] > 50].index)
df = df.drop(columns=missing_columns)

In [9]:
# Удалим выбросы
df = df[df['Site EUI (kBtu/ft²)'] < 1000000]

In [10]:
# Удалим значения с пропущенными метками
df = df[df['ENERGY STAR Score'].notnull()]

## Добавляем признаки

In [11]:
# Разделим столбец с датой application_dt на несколько признаков, включая год, месяц, день и т.д.
def split_date(df):
    df["Generation Date"] = pd.to_datetime(df["Generation Date"])
    df['Generation_year'] = df['Generation Date'].apply(lambda x: x.year)
    df['Generation_month'] = df['Generation Date'].apply(lambda x: x.month)
    df['Generation_day'] = df['Generation Date'].apply(lambda x: x.day)
    return df.drop(['Generation Date'], axis=1)

df = split_date(df)

### Преобразуем текст в числовые признаки

In [12]:
for c in df.columns:
    if df[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(df[c].values))
        df[c] = lbl.transform(list(df[c].values))

### Разделим данные на тренировочные и тестовые

In [13]:
X = df.drop(['ENERGY STAR Score'], axis=1)
y = df['ENERGY STAR Score']

In [14]:
imputer = Imputer(strategy='median')

imputer.fit(X)

X = imputer.transform(X)

scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X)

X = scaler.transform(X)

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=41)

## Обучим модели

In [16]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(random_state=41)
model.fit(X_train, y_train)
mae(y_test, model.predict(X_test))

9.337561992041689

In [17]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(random_state=41)
model.fit(X_train, y_train)
mae(y_test, model.predict(X_test))

8.4483330083837

In [296]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=41)
model.fit(X_train, y_train)

mae(y_test, model.predict(X_test))

6.9764029700396435

В итоге, лучший результат набрал RandomForestRegressor